In [25]:
import sys
sys.path.append(r'C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis')
sys.path.append(r'C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\0grafana2pgsql\pgsql_set')
from src.ToolPGSQL import ToolPGSQL

In [47]:
a="$aaa $bbb"

In [49]:
from string import Template
Template(a).safe_substitute(aaa='bbb')

'bbb $bbb'

In [41]:
import json
with open(r'C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\0grafana2pgsql\pgsql_set\daily_check_v2.json','r') as f:
    key_sqls=json.loads(f.read())

{'DAU (R1)': 'SELECT\n    \'#Overall#\' as "App", \n    1 as "Rolling",\n    Overall as "DAU", \n    Overall-LastOverall as "WoW -7d", (Overall-LastOverall)/LastOverall as "WoW% -7d", \n    Overall-Last2_Overall as "WoW -14d", (Overall-Last2_Overall)/Last2_Overall as "WoW% -14d"\nFROM (\n    SELECT  \n        sum(case when metrics_date = last_date then Overall else 0 end) as Overall,\n        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + \'-7 d\' as date), \'yyyy-mm-dd\') then Overall else 0 end) as LastOverall,\n        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + \'-14 d\' as date), \'yyyy-mm-dd\') then Overall else 0 end) as Last2_Overall\n    FROM (\n        SELECT \n            metrics_date,\n            CASE WHEN cast(${__to:date:\'YYYY-MM-DD\'} as TEXT)>=\'${DauMetricsDate}\' THEN cast(\'${DauMetricsDate}\' as TEXT) ELSE cast(${__to:date:\'YYYY-MM-DD\'} as TEXT) END as last_date,\n            SUM(CAST(metrics->\'ActiveUser\'

In [40]:
pgsql_transform_table={"$__timeFilter(metrics_date)":"""
           metrics_date >= '$start_date' -- Adjust the start date as needed
    AND metrics_date <= '$end_date' -- Adjust the end date as needed""",
           "metrics_date::date":"metrics_date",
           """${__from:date:'YYYY-MM-DD'}""":"'$start_date'",
           """${__to:date:'YYYY-MM-DD'}""":"'$end_date'",
           """${Applications}""":"""'2130688B018F4B44BBED68E7A42BBA1E',
        'AE427635ADC245AE973038BCB3D7C21B',
        '4DC5714ABCAD449BA13A9B701A3CF296',
        '4A5B528B59954AAE8725B509A41FBF1A',
        'F185A93DE6764B098D55089F610A3FB8',
        'FC320C411FC12CD4DFBE9A00F3161364'""",
        """${DauMetricsDate}""":"""${dau_metrics_date}"""
           }

In [42]:
def regular(value:str):
    for origin,target in pgsql_transform_table.items():
        value=value.replace(origin,target)
    return value
toolPGSQL = ToolPGSQL(timeout=20)
for key,value in key_sqls.items():
    key_sqls[key]=regular(value)
from string import Template
# for key,sql_template in key_sqls.items():
#     print(toolPGSQL.execute_v2(Template(sql_template).substitute(start_date='2024-07-01',end_date='2024-07-16',dau_metrics_date='2024-07-16',rolling_day=1),debug=False))

init_SQL_Tool, time_out=20s
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
<connection object at 0x0000018F79F8F590; dsn: 'user=bingviz_metrics password=xxx connect_timeout=20 dbname=metrics host=bingviz-metrics-storage-dup.postgres.database.azure.com port=5432', closed: 0>


In [44]:
print(str(key_sqls))

{'DAU (rolling ${rolling_day} day)': 'select metrics_date as "time", AVG("DAU") OVER (ORDER BY metrics_date ROWS BETWEEN ${rolling_day}-1 PRECEDING AND CURRENT ROW) AS "DAU (R${rolling_day})"\nfrom \n(\nSELECT \n    metrics_date, SUM(CAST(metrics->\'ActiveUser\'->\'uu\'->>\'uu\' AS DOUBLE PRECISION)) as "DAU"\nFROM sapphire.sapphire_engagement_metrics_master\nWHERE metrics_date >= to_char(cast(cast(\'$start_date\' as timestamp) + \'- ${rolling_day} DAY\' as date), \'yyyy-mm-dd\')::TEXT AND metrics_date <= \'$end_date\'\n      AND application_id IN (\'2130688B018F4B44BBED68E7A42BBA1E\',\n        \'AE427635ADC245AE973038BCB3D7C21B\',\n        \'4DC5714ABCAD449BA13A9B701A3CF296\',\n        \'4A5B528B59954AAE8725B509A41FBF1A\',\n        \'F185A93DE6764B098D55089F610A3FB8\',\n        \'FC320C411FC12CD4DFBE9A00F3161364\')\n      AND period = 1\n      AND market = \'#Overall#\'\n      AND os_version = \'#Overall#\'\n      AND device_model = \'#Overall#\'\n      AND client_version = \'#Overall

In [30]:
key_sqls.keys()

dict_keys(['DAU (rolling ${rolling_day} day)', 'User Retention 7d rolling ${rolling_day} day - all markets', 'Chat Active DAU (rolling ${rolling_day} day)'])

In [35]:
from string import Template
from datetime import datetime, timedelta
from datetime import datetime, timedelta
 
def tool_dau(start_date:str,end_date:str,rolling_day=1):
    """Retrieve DAU within the [start_date, end_date] time range, optionally using a rolling average over rolling_day days.
    
    Input:
        start_date (e.g., 2024-07-10), end_date (e.g., 2024-07-14) ,rolling_day (e.g. 7 or 1)
    Output:
        a pandas dataframe
    """
    sql_template=key_sqls["DAU (rolling ${rolling_day} day)"]
    return_pandas=toolPGSQL.execute_v2(Template(sql_template).substitute(start_date=start_date,end_date=end_date,rolling_day=rolling_day),debug=False)
    return return_pandas
def tool_retention(start_date:str,end_date:str,rolling_day=7):
    """Retrieve Chat DAU within the [start_date, end_date] time range, optionally using a rolling average over rolling_day days.
    
    Input:
        start_date (e.g., 2024-07-10), end_date (e.g., 2024-07-14) ,rolling_day (e.g. 7 or 1)
    Output:
        a pandas dataframe
    """
    sql_template=key_sqls['User Retention 7d rolling ${rolling_day} day - all markets']
    return_pandas=toolPGSQL.execute_v2(Template(sql_template).substitute(start_date=start_date,end_date=end_date,dau_metrics_date=end_date,rolling_day=rolling_day),debug=False)
    return return_pandas
    
def tool_chat_dau(start_date:str,end_date:str,rolling_day=1):
    """Retrieve Chat DAU within the [start_date, end_date] time range, optionally using a rolling average over rolling_day days.
    
    Input:
        start_date (e.g., 2024-07-10), end_date (e.g., 2024-07-14) ,rolling_day (e.g. 7 or 1)
    Output:
        a pandas dataframe
    """
    sql_template=key_sqls["Chat Active DAU (rolling ${rolling_day} day)"]
    return_pandas=toolPGSQL.execute_v2(Template(sql_template).substitute(start_date=start_date,end_date=end_date,dau_metrics_date=end_date,rolling_day=rolling_day),debug=False)
    return return_pandas
check_date="2024-07-11"
def daily_check(check_date:str="2024-07-11"):
    """Retrieve daily check results for [check_date], providing metrics for DAU, retention, and Chat DAU.
    
    Input:
        check_date (e.g., 2024-07-11)
    Output:
        a formatted string containing daily check results
    """
    # last date having data

    res_list=[]
    tool_functions= [tool_dau,tool_retention,tool_chat_dau]
    for tool in tool_functions:
        metrics_date=tool(start_date=(datetime.strptime(check_date, '%Y-%m-%d') -  timedelta(days=7)).strftime('%Y-%m-%d'),end_date=check_date,rolling_day=1).iloc[-1].loc["time"]
        seven_days_ago=(datetime.strptime(metrics_date, '%Y-%m-%d') -  timedelta(days=7)).strftime('%Y-%m-%d')
        print(metrics_date,seven_days_ago)
        data_r1=tool(start_date=seven_days_ago,end_date=metrics_date,rolling_day=1)
        r1_wow_change=data_r1.loc[data_r1['time']==metrics_date].iloc[-1,-1]-data_r1.loc[data_r1['time']==seven_days_ago].iloc[-1,-1]

        data_r7=tool(start_date=seven_days_ago,end_date=metrics_date,rolling_day=7)
        r7_wow_change=data_r7.loc[data_r7['time']==metrics_date].iloc[-1,-1]-data_r7.loc[data_r7['time']==seven_days_ago].iloc[-1,-1]
        res_list.append([metrics_date,data_r7.iloc[-1,-1],r7_wow_change,data_r1.iloc[-1,-1],r1_wow_change])

    
    # print(res_list)
    daily_template=f"""Check Date: {check_date}""";
    
    daily_template+=f"""\n Metrics Date: {res_list[0][0]} DAU: avg-7d {res_list[0][1]/1000000:.3f} M ({res_list[0][2]/1000:.3f}k WoW), last-1d {res_list[0][3]/1000000:.3f} M ({res_list[0][4]/1000:.3f}k WoW)"""
    daily_template+=f"""\n Metrics Date: {res_list[1][0]} New Retention Day7: avg-7d {res_list[1][1]*100:.1f}% ({res_list[1][2]*100:.2f} pp WoW), last-1d {res_list[1][3]*100:.3f}% ({res_list[1][4]*100:.2f}pp WoW)"""
    daily_template+=f"""\n Metrics Date: {res_list[2][0]} Chat DAU: avg-7d {res_list[2][1]/1000000:.3f} M ({res_list[2][2]/1000:.3f}k WoW), last-1d {res_list[2][3]/1000000:.3f} M ({res_list[2][4]/1000:.3f} K WoW)"""
    return daily_template
# DAU: avg-7d 5.13 M (-193.0k WoW), last-1d 5.21 M (-46.2k WoW)
# New Retention Day7: avg-7d 10.54% (+2.66pp WoW), last-1d 12.29% (+4.23pp WoW)
# Chat DAU: avg-7d 1.13 M (-64.7k WoW), last-1d 1.14 M (-74.5 K WoW)"""


[['2024-07-11', 5133543.0, -192994.85714285728, 5211443.0, -46197.0],
  ['2024-07-10', 0.12328767072713977, 0.02388727949170459, 0.12913746749526203, 0.004080515735680468],
  ['2024-07-11', 1116056.4285714286, -69815.71428571432, 1081294.0, -116028.0]]

[['2024-07-11', 5133543.0, -192994.85714285728, 5211443.0, -46197.0],
 ['2024-07-10',
  0.12328767072713977,
  0.02388727949170459,
  0.12913746749526203,
  0.004080515735680468],
 ['2024-07-11', 1116056.4285714286, -69815.71428571432, 1081294.0, -116028.0]]

In [37]:
print(daily_check(check_date="2024-07-01"))

Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
Reconnected to the database
2024-07-01 2024-06-24
Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
Reconnected to the database
Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
Reconnected to the database
Connection closed, reconnecting...
Data

In [75]:
seven_days_ago=(datetime.strptime(check_date, '%Y-%m-%d') -  timedelta(days=7)).strftime('%Y-%m-%d')
dau_r1=tool_retention(start_date=seven_days_ago,end_date=check_date,rolling_day=1)
dau_r1

Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
Reconnected to the database


C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\src\ToolPGSQL.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(sql_query,self.conn)


,time,Retention D7 R1
0,2024-07-04,0.122877
1,2024-07-05,0.122118
2,2024-07-06,0.122176
3,2024-07-07,0.123089
4,2024-07-08,0.121755
5,2024-07-09,0.121860
6,2024-07-10,0.129137


In [69]:


print()

Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
Reconnected to the database


IndexError: index -1 is out of bounds for axis 0 with size 0

In [52]:
print(dau_r1)

         time   DAU (R1)
0  2024-07-10  5232876.0
1  2024-07-11  5211443.0


In [50]:
dau_r1.iloc[-1,-1]-dau_r1.iloc[-2,-1]

-21433.0

In [40]:
daily_check("2024-07-11")

Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
Reconnected to the database


C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\src\ToolPGSQL.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(sql_query,self.conn)


         time   DAU (R1)
0  2024-07-10  5232876.0
1  2024-07-11  5211443.0
Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
Reconnected to the database


C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\src\ToolPGSQL.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(sql_query,self.conn)


         time      DAU (R7)
0  2024-07-04  5.257640e+06
1  2024-07-05  5.220003e+06
2  2024-07-06  5.123782e+06
3  2024-07-07  5.065069e+06
4  2024-07-08  5.097929e+06
5  2024-07-09  5.124687e+06
6  2024-07-10  5.140143e+06
7  2024-07-11  5.133543e+06


In [35]:
tool_dau("2024-07-10","2024-07-12",1)

Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 20}
Reconnected to the database


C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\src\ToolPGSQL.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(sql_query,self.conn)


,time,DAU (R1)
0,2024-07-09,5258479.0
1,2024-07-10,5232876.0
2,2024-07-11,5211443.0
3,2024-07-12,5079268.0
